# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()


In [ ]:
## 1. 모듈 import
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate

## [방법 1]
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.chat_history import InMemoryChatMessageHistory

## [방법 2]
# from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory

## [방법 3]
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
## type: dict 
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_completion_tokens=150,
)

## 7. chain 구성
chain = prompt | llm 

## 8. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 9. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '1234'}}      
    )

In [2]:
get_history('1234')

InMemoryChatMessageHistory(messages=[HumanMessage(content='이력서 작성을 어떻게 해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='이력서 작성 방법에 대해 안내해 드리겠습니다. 효과적인 이력서는 자신의 경력과 역량을 명확하고 간결하게 전달하여 채용 담당자의 관심을 끄는 것이 중요합니다. 아래 단계별로 작성 방법을 설명드리겠습니다.\n\n1. 기본 정보\n- 이름, 연락처(전화번호, 이메일), 주소(선택사항)를 명확히 기재하세요.\n- 최근 연락이 가능한 연락처를 기입하는 것이 좋습니다.\n\n2. 자기소개 또는 경력개요\n- 간단하게 본인의 전문성, 강점, 지원 동기를 요약하세요.\n- 해당 직무에 적합한 역량을 강조하는 것이 좋습니다.\n\n3.', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-335e9686-c8de-41d1-82f8-16a4b91223d6-0'), HumanMessage(content='이력서 작성해주세요!', additional_kwargs={}, response_metadata={}), AIMessage(content='물론입니다! 지원자의 기본 정보, 경력 사항, 학력, 기술 및 자격증, 그리고 자기소개 등을 포함한 예시 이력서를 아래에 작성해 드리겠습니다. 참고용으로 활용하시거나 필요에 맞게 수정하시면 됩니다.\n\n---\n\n**이력서**\n\n**개인정보**  \n이름: 홍길동  \n생년월일: 1990년 5월 15일  \n연락처: 010-1234-5678  \n이메일: honggildong@email.com  \n주소: 서울시 강남구 테헤란로 123  \n\n---\n

In [3]:
for info in get_history('1234').messages:
    print(info.content)
    print('*' * 50)

이력서 작성을 어떻게 해야 하나요?
**************************************************
이력서 작성 방법에 대해 안내해 드리겠습니다. 효과적인 이력서는 자신의 경력과 역량을 명확하고 간결하게 전달하여 채용 담당자의 관심을 끄는 것이 중요합니다. 아래 단계별로 작성 방법을 설명드리겠습니다.

1. 기본 정보
- 이름, 연락처(전화번호, 이메일), 주소(선택사항)를 명확히 기재하세요.
- 최근 연락이 가능한 연락처를 기입하는 것이 좋습니다.

2. 자기소개 또는 경력개요
- 간단하게 본인의 전문성, 강점, 지원 동기를 요약하세요.
- 해당 직무에 적합한 역량을 강조하는 것이 좋습니다.

3.
**************************************************
이력서 작성해주세요!
**************************************************
물론입니다! 지원자의 기본 정보, 경력 사항, 학력, 기술 및 자격증, 그리고 자기소개 등을 포함한 예시 이력서를 아래에 작성해 드리겠습니다. 참고용으로 활용하시거나 필요에 맞게 수정하시면 됩니다.

---

**이력서**

**개인정보**  
이름: 홍길동  
생년월일: 1990년 5월 15일  
연락처: 010-1234-5678  
이메일: honggildong@email.com  
주소: 서울시 강남구 테헤란로 123  

---

**자기소개 및 경력개요**  
신뢰성과 성실성을 바탕으로 문제
**************************************************
이름: 박보검, 지원분야: 웹 개발 신입, 보유 능력: Python, HTML, CSS, LLM
**************************************************
물론입니다! 아래는 지원자 박보검 님의 "웹 개발 신입" 분야에 적합한 이력서 예시입니다. 자연스럽고 전문적인 느낌을 살려서 작성하였으며, 필요에 따라 자유롭게 수정